In [ ]:
# Cell 1: Import Dependencies & Setup
import json
import logging
import sqlite3
import time
import psutil
import requests
import secrets
from http.server import BaseHTTPRequestHandler, HTTPServer

# --- Configuration ---
PORT = 5001
GO_SERVICE_URL = "http://127.0.0.1:8080/verify"
DB_FILE = "commitments.db"

# --- Logging Setup ---
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

print(f"--- Server Configured on Port {PORT} ---")
print(f"--- Upstream ZKP Verifier: {GO_SERVICE_URL} ---")

In [ ]:
# Cell 2: Database Management Functions

def get_db_connection():
    """Creates a connection to the database with WAL mode enabled."""
    conn = sqlite3.connect(DB_FILE)
    conn.execute('PRAGMA journal_mode=WAL;') # Write-Ahead Logging for concurrency
    return conn

def init_db():
    """Initializes the database schema if it doesn't exist."""
    conn = get_db_connection()
    cursor = conn.cursor()
    # Updated to include salt storage for Argon2id compatibility
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS commitments (
            id INTEGER PRIMARY KEY,
            username TEXT NOT NULL UNIQUE,
            commitment TEXT NOT NULL,
            salt TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()
    logging.info("Database initialized with salt storage.")

def reset_db():
    """Truncates the commitments table. Used for testing."""
    try:
        conn = get_db_connection()
        conn.execute("DELETE FROM commitments")
        conn.commit()
        conn.close()
        logging.info("Database reset (all rows cleared).")
        return True
    except sqlite3.Error as e:
        logging.error(f"Database Error during reset: {e}")
        return False

def save_commitment(username, commitment, salt):
    """Stores a new User -> Commitment + Salt mapping."""
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(
            'INSERT INTO commitments (username, commitment, salt) VALUES (?, ?, ?)', 
            (username, commitment, salt)
        )
        conn.commit()
    except sqlite3.IntegrityError:
        return False # User already exists
    finally:
        conn.close()
    return True

def get_stored_commitment(username):
    """Retrieves the commitment string for a specific user."""
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT commitment FROM commitments WHERE username = ?', (username,))
    row = cursor.fetchone()
    conn.close()
    return row[0] if row else None

def get_user_data(username):
    """Retrieves both commitment and salt for a specific user."""
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT commitment, salt FROM commitments WHERE username = ?', (username,))
    row = cursor.fetchone()
    conn.close()
    if row:
        return {"commitment": row[0], "salt": row[1]}
    return None

# Initialize immediately on cell run
init_db()

In [ ]:
# Cell 3: Helper Functions (Metrics & ZKP)

def log_metrics(start_time, url):
    """Logs request duration and system resource usage."""
    elapsed = time.time() - start_time
    cpu = psutil.cpu_percent(interval=None)
    ram = psutil.virtual_memory().percent
    logging.info(f"[{url}] took {elapsed:.4f}s | Sys: CPU {cpu}% RAM {ram}%")

def verify_proof_upstream(proof, public_witness):
    """
    Forwards the Proof and Public Witness to the Go Service.
    The Go Service performs the actual BN254 Elliptic Curve pairing check.
    """
    payload = {
        "proof": proof,
        "public_witness": public_witness
    }
    
    try:
        # We set a strict timeout to ensure responsiveness
        response = requests.post(GO_SERVICE_URL, json=payload, timeout=5)
        
        if response.status_code == 200:
            result = response.json()
            is_valid = result.get('valid', False)
            logging.info(f"Go Verification Result: {is_valid}")
            return is_valid
        else:
            logging.error(f"Go Service Error: {response.status_code} - {response.text}")
            return False
    except Exception as e:
        logging.error(f"Failed to contact Go Service: {e}")
        return False

print("--- Logic Layer Ready ---")

In [ ]:
# Cell 4: HTTP Request Handler & Server Loop (Secure Witness Construction)

active_challenges = set()

class RequestHandler(BaseHTTPRequestHandler):
    
    def _send_json(self, code, data):
        self.send_response(code)
        self.send_header('Content-type', 'application/json')
        self.end_headers()
        self.wfile.write(json.dumps(data).encode())

    def do_GET(self):
        if self.path == '/challenge':
            # Generate a fresh 64-bit nonce
            nonce = str(secrets.randbits(64))
            active_challenges.add(nonce)
            self._send_json(200, {'challenge': nonce})
        else:
            self.send_error(404)

    def do_POST(self):
        try:
            content_length = int(self.headers.get('Content-Length', 0))
            body = self.rfile.read(content_length).decode('utf-8')
            data = json.loads(body) if body else {}

            if self.path == '/enroll':
                self.handle_enroll(data)
            elif self.path == '/verifyCommitment':
                self.handle_verify(data)
            elif self.path == '/reset':
                self.handle_reset()
            else:
                self.send_error(404)
        except Exception as e:
            logging.error(f"Server Error: {e}")
            self._send_json(500, {"error": str(e)})

    def handle_reset(self):
        active_challenges.clear()
        if reset_db():
            self._send_json(200, {"message": "Reset success"})
        else:
            self._send_json(500, {"error": "Reset failed"})

    def handle_enroll(self, data):
        username = data.get('username')
        # [NEW] We now expect the raw commitment string, not the witness blob
        commitment = data.get('commitment') 
        salt = data.get('salt') # [FIXED] Extract salt from request

        if not username or not commitment or not salt:
            return self._send_json(400, {"error": "Missing fields"})

        # [FIXED] Pass salt to save_commitment
        if save_commitment(username, commitment, salt):
            logging.info(f"User enrolled: {username}")
            self._send_json(201, {"message": "Enrollment successful"})
        else:
            self._send_json(409, {"error": "User already exists"})

    def handle_verify(self, data):
        username = data.get('username')
        proof = data.get('proof')
        challenge_response = data.get('challenge')

        if not username or not proof or not challenge_response:
            return self._send_json(400, {"error": "Missing fields"})

        # 1. Nonce Check
        if challenge_response not in active_challenges:
            logging.warning(f"Replay/Expire for {username}")
            return self._send_json(403, {"error": "Invalid/Expired Challenge"})
        
        active_challenges.remove(challenge_response)

        # 2. Retrieve Stored Commitment (Identity Source of Truth)
        stored_commitment = get_stored_commitment(username)
        if not stored_commitment:
            return self._send_json(403, {"error": "User not found"})

        # 3. Secure Verification Check
        # Prevents attempts to bypass binding with null or default nonces
        if not challenge_response or challenge_response == "0":
             return self._send_json(403, {"error": "Illegal Challenge Value"})

        payload = {
            "proof": proof,
            "commitment": stored_commitment,
            "challenge": challenge_response
        }
        
        try:
            response = requests.post(GO_SERVICE_URL, json=payload, timeout=5)
            if response.status_code == 200 and response.json().get('valid'):
                logging.info(f"Auth Success: {username}")
                self._send_json(200, {"message": "Login Successful"})
            else:
                logging.warning(f"Auth Failed: {username}")
                self._send_json(403, {"error": "Invalid ZKP (Proof does not match User/Challenge)"})
        except Exception as e:
            logging.error(f"Go Service Fail: {e}")
            self._send_json(500, {"error": "Internal Verifier Error"})

def run():
    server_address = ('127.0.0.1', PORT)
    try:
        httpd = HTTPServer(server_address, RequestHandler)
        print(f'>>> Authentication Server running on http://127.0.0.1:{PORT} <<<')
        httpd.serve_forever()
    except OSError as e:
        print(f"Server Start Error: {e}")

if __name__ == '__main__':
    run()